In [3]:

# -----------------------------------------------------------------------------
# region import packages

# management
import glob
import pickle
import warnings
warnings.filterwarnings('ignore')
import os
import sys  # print(sys.path)
sys.path.append('/work/ollie/qigao001')
sys.path.append('/home/users/qino')
os.chdir('/home/users/qino')
# data analysis
import numpy as np
import xarray as xr
import dask
dask.config.set({"array.slicing.split_large_chunks": True})
# from dask.diagnostics import ProgressBar
# pbar = ProgressBar()
# pbar.register()
from scipy import stats
import xesmf as xe
import pandas as pd
from metpy.interpolate import cross_section
from statsmodels.stats import multitest
import pycircstat as circ
from scipy.stats import circstd
import cmip6_preprocessing.preprocessing as cpp

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import cm
import cartopy.crs as ccrs
plt.rcParams['pcolor.shading'] = 'auto'
mpl.rcParams['figure.dpi'] = 600
mpl.rc('font', family='Times New Roman', size=10)
mpl.rcParams['axes.linewidth'] = 0.2
plt.rcParams.update({"mathtext.fontset": "stix"})
import matplotlib.animation as animation
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator
import cartopy.feature as cfeature

# self defined
from a_basic_analysis.b_module.mapplot import (
    globe_plot,
    hemisphere_plot,
    quick_var_plot,
    mesh2plot,
    framework_plot1,
    remove_trailing_zero,
    remove_trailing_zero_pos,
)

from a_basic_analysis.b_module.basic_calculations import (
    mon_sea_ann,
    regrid,
    find_ilat_ilon,
)

from a_basic_analysis.b_module.namelist import (
    month,
    month_num,
    month_dec,
    month_dec_num,
    seasons,
    seasons_last_num,
    hours,
    months,
    month_days,
    zerok,
    panel_labels,
    seconds_per_d,
)

from a_basic_analysis.b_module.source_properties import (
    source_properties,
    calc_lon_diff,
)

from a_basic_analysis.b_module.statistics import (
    fdr_control_bh,
    check_normality_3d,
    check_equal_variance_3d,
    ttest_fdr_control,
)

from a_basic_analysis.b_module.component_plot import (
    cplot_ice_cores,
    plt_mesh_pars,
)

# endregion
# -----------------------------------------------------------------------------



In [4]:

#-------- import EC reconstruction
ec_sst_rec = {}
# 47 cores
ec_sst_rec['original'] = pd.read_excel(
    'data_sources/LIG/mmc1.xlsx',
    sheet_name='Capron et al. 2017', header=0, skiprows=12, nrows=47,
    # usecols=['Station', 'Latitude', 'Longitude', 'Area', 'Type',
    #          '127 ka Median PIAn [°C]', '127 ka 2s PIAn [°C]'],
    )

# 2 cores
ec_sst_rec['SO_ann'] = ec_sst_rec['original'].loc[
    (ec_sst_rec['original']['Area']=='Southern Ocean') & \
        (ec_sst_rec['original']['Type']=='Annual SST'),]
# 15 cores
ec_sst_rec['SO_djf'] = ec_sst_rec['original'].loc[
    (ec_sst_rec['original']['Area']=='Southern Ocean') & \
        (ec_sst_rec['original']['Type']=='Summer SST'),]
# 4 cores
ec_sst_rec['AIS_am'] = ec_sst_rec['original'].loc[
    ec_sst_rec['original']['Area']=='Antarctica',]


#-------- import dc reconstructions
# 30 cores
dc_sst_rec = {}
dc_sst_rec['original'] = pd.read_csv(
    'data_sources/LIG/Chandler-Langebroek_2021_SST-anom.tab',
    sep='\t', header=0, skiprows=76,
    )
dc_sst_rec['summary'] = pd.read_csv(
    'data_sources/LIG/Chandler-Langebroek_2021_SST-anom-stats.tab',
    sep='\t', header=0, skiprows=25,
    )
# 25 cores
dc_sst_rec['ann'] = dc_sst_rec['original'].loc[
    (dc_sst_rec['original']['Season (A annual, S summer (months JFM))'] == 'A')
]
# 26 cores
dc_sst_rec['sum'] = dc_sst_rec['original'].loc[
    (dc_sst_rec['original']['Season (A annual, S summer (months JFM))'] == 'S')
]

dc_sst_rec['ann_124'] = dc_sst_rec['ann'].loc[dc_sst_rec['ann']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 124]
dc_sst_rec['ann_126'] = dc_sst_rec['ann'].loc[dc_sst_rec['ann']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 126]
dc_sst_rec['ann_128'] = dc_sst_rec['ann'].loc[dc_sst_rec['ann']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 128]
dc_sst_rec['ann_130'] = dc_sst_rec['ann'].loc[dc_sst_rec['ann']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 130]

dc_sst_rec['sum_124'] = dc_sst_rec['sum'].loc[dc_sst_rec['sum']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 124]
dc_sst_rec['sum_126'] = dc_sst_rec['sum'].loc[dc_sst_rec['sum']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 126]
dc_sst_rec['sum_128'] = dc_sst_rec['sum'].loc[dc_sst_rec['sum']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 128]
dc_sst_rec['sum_130'] = dc_sst_rec['sum'].loc[dc_sst_rec['sum']['Age [ka BP] (Chronology follows Lisiecki a...)'] == 130]




In [29]:
dc_sst_rec['summary'].loc[dc_sst_rec['summary']['Age [ka BP]'] == 124]

,Age [ka BP],"NOBS [#] (Number of records, annual SST)","SST anomaly [°C] (Mean annual SST anomaly, rela...)",SST anomaly std dev [±] (Annual SST anomaly),"CI (95% CI, annual SST anomaly, degC)","NOBS [#] (Number of records, summer (JF...)",SST anomaly [°C] (Mean summer (JFM) SST anomaly...),SST anomaly std dev [±] (Summer (JFM) SST anomaly),"CI (95% CI, summer (JFM) SST anom...)"
62,124,15,0.851939,1.8619,1.031086,16,1.300225,1.985543,1.058021


In [30]:
dc_sst_rec['ann_124']['SST anomaly [°C] (Anomaly relative to the World...)'].count()

16

In [31]:
np.nanmean(dc_sst_rec['ann_124']['SST anomaly [°C] (Anomaly relative to the World...)'])

0.8315865312500001

In [32]:
dc_sst_rec['sum_124']['SST anomaly [°C] (Anomaly relative to the World...)'].count()

17

In [33]:
np.nanmean(dc_sst_rec['sum_124']['SST anomaly [°C] (Anomaly relative to the World...)'])

1.2017422294117648